In [1]:


import pandas as pd
import numpy as np 
from sklearn import preprocessing
import xgboost as xgb
from sklearn.feature_extraction import DictVectorizer


def xgboost_pred(train,labels,test):
    params = {}
    params["objective"] = "reg:linear"
    params["eta"] = 0.005
    params["min_child_weight"] = 5
    params["subsample"] = 0.8
    params["colsample_bytree"] = 0.8
    params["scale_pos_weight"] = 0.8
    params["silent"] = 1
    params["max_depth"] = 10

    plst = list(params.items())

    #Using 5000 rows for early stopping. 
    offset = 4000

    num_rounds = 10000
    xgtest = xgb.DMatrix(test)

    #create a train and validation dmatrices 
    xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
    xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])
    #	xgtrain = xgb.DMatrix(train, label=labels)
    #xgval = xgb.DMatrix(train, label=labels)

    #train using early stopping and predict
    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=100)
    #model = xgb.train(plst, xgtrain, 1000)
    preds1 = model.predict(xgtest)


    #reverse train and labels and use different 5k for early stopping. 
    # this adds very little to the score but it is an option if you are concerned about using all the data. 
    train = train[::-1,:]
    labels = np.log(labels[::-1])

    xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
    xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])
    #	xgtrain = xgb.DMatrix(train, label=labels)


    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=100)
    #	model = xgb.train(plst, xgtrain, 1000)
    preds2 = model.predict(xgtest)


    #combine predictions
    #since the metric only cares about relative rank we don't need to average
    preds = preds1*2.6 + preds2*7.4
    return preds

#load train and test 
train  = pd.read_csv('../data/train.csv', index_col=0)
test  = pd.read_csv('../data/test.csv', index_col=0)


labels = train.Hazard
train.drop('Hazard', axis=1, inplace=True)

train_s = train
test_s = test


train_s.drop('T2_V10', axis=1, inplace=True)
train_s.drop('T2_V7', axis=1, inplace=True)
train_s.drop('T1_V13', axis=1, inplace=True)
train_s.drop('T1_V10', axis=1, inplace=True)

test_s.drop('T2_V10', axis=1, inplace=True)
test_s.drop('T2_V7', axis=1, inplace=True)
test_s.drop('T1_V13', axis=1, inplace=True)
test_s.drop('T1_V10', axis=1, inplace=True)

columns = train.columns
test_ind = test.index


train_s = np.array(train_s)
test_s = np.array(test_s)

# label encode the categorical variables
for i in range(train_s.shape[1]):
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_s[:,i]) + list(test_s[:,i]))
    train_s[:,i] = lbl.transform(train_s[:,i])
    test_s[:,i] = lbl.transform(test_s[:,i])

train_s = train_s.astype(float)
test_s = test_s.astype(float)


preds1 = xgboost_pred(train_s,labels,test_s)

#model_2 building

train = train.T.to_dict().values()
test = test.T.to_dict().values()

vec = DictVectorizer()
train = vec.fit_transform(train)
test = vec.transform(test)

preds2 = xgboost_pred(train,labels,test)


preds = 0.6 * preds1 + 0.4 * preds2

#generate solution
preds = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds = preds.set_index('Id')
preds.to_csv('../output/xgboost_benchmark_kk.csv')

Will train until val error hasn't decreased in 100 rounds.
[0]	train-rmse:5.335824	val-rmse:5.283714
[1]	train-rmse:5.321445	val-rmse:5.270116
[2]	train-rmse:5.307376	val-rmse:5.256952
[3]	train-rmse:5.293237	val-rmse:5.243780
[4]	train-rmse:5.279428	val-rmse:5.231208
[5]	train-rmse:5.265477	val-rmse:5.218444
[6]	train-rmse:5.251749	val-rmse:5.205625
[7]	train-rmse:5.238087	val-rmse:5.193036
[8]	train-rmse:5.224582	val-rmse:5.180253
[9]	train-rmse:5.210925	val-rmse:5.167638
[10]	train-rmse:5.197494	val-rmse:5.155128
[11]	train-rmse:5.184205	val-rmse:5.142974
[12]	train-rmse:5.170910	val-rmse:5.130790
[13]	train-rmse:5.157525	val-rmse:5.118582
[14]	train-rmse:5.144433	val-rmse:5.106532
[15]	train-rmse:5.131371	val-rmse:5.094603
[16]	train-rmse:5.118539	val-rmse:5.082829
[17]	train-rmse:5.105578	val-rmse:5.070930
[18]	train-rmse:5.092763	val-rmse:5.059136
[19]	train-rmse:5.080469	val-rmse:5.047732
[20]	train-rmse:5.067969	val-rmse:5.036310
[21]	train-rmse:5.055401	val-rmse:5.024804
[22]	